<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=56214152a9f93b4aac6fc6ef9af9f8c20d675987160486dc48a7ffe8cb729975
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-12 11:17:28
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 18.02 K (0.13%)
Current PnL: -17.57 L (-11.82%)
CY Booked + Current PnL: -6.54 L (-4.4%)
-------------------
Total profit:  2.08 L
Total loss:  -19.65 L
-------------------
Total Booked + Current PnL: 20.44 L (16.8%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.67 L (59.08%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.42,59.0,H-LC,11.17,172964.0,16787.0,9305.0,-0.22,10.75,5.38,16.71,22.0,1.80,1.26,30.06,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,60.0,M-LC,4.26,218002.0,16522.0,10682.0,-0.12,8.20,4.90,13.51,60.0,1.55,1.59,21.83,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,101.92,53.0,M-SC,5.01,86971.0,-13806.0,13898.0,-1.37,-13.70,15.98,0.09,245.0,-0.99,0.63,14.26,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
41,ITC,452.00,-37.08,57.0,H-LC,2.96,203424.0,3286.0,17596.0,0.23,1.64,8.65,10.44,4.0,0.19,1.48,7.30,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,64.0,M-MC,6.95,227183.0,2221.0,165435.0,0.08,0.99,72.82,74.53,192.0,0.01,1.65,33.56,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-21.57,39.0,H-MC,2.09,135024.0,-764.0,43518.0,-0.17,-0.56,32.23,31.48,107.0,-0.02,0.98,11.49,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-1.08,56.0,H-MC,1.67,245341.0,-2390.0,77454.0,-0.11,-0.96,31.57,30.30,92.0,-0.03,1.79,12.20,X40,ATH,ELECTRICAL
12,BANKINDIA,190.00,-30.43,62.0,H-MC,5.63,180715.0,907.0,111501.0,-0.51,0.50,61.70,62.52,88.0,0.01,1.31,34.76,XR,NTT,BANKS
51,PGHH,17973.08,-31.08,51.0,H-MC,4.53,199665.0,-1155.0,69923.0,0.05,-0.58,35.02,34.25,80.0,-0.02,1.45,4.59,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.73,37.0,H-LC,6.04,300051.0,-15791.0,128452.0,-0.92,-5.00,42.81,35.67,5.0,-0.12,2.18,8.16,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.47,44.0,H-LC,13.52,267632.0,267.0,65195.0,0.16,0.10,24.36,24.48,48.0,0.00,1.95,55.13,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.09,47.0,H-LC,8.06,225148.0,-8996.0,57615.0,-0.76,-3.84,25.59,20.77,86.0,-0.16,1.64,2.81,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.12,229848.0,-12076.0,75344.0,-0.25,-4.99,32.78,26.15,8.0,-0.16,1.67,8.62,X40,ATH,IT
56,RELIANCE,1533.00,-13.29,52.0,H-LC,6.04,217495.0,7129.0,21663.0,0.79,3.39,9.96,13.68,37.0,0.33,1.58,20.47,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.47,59.0,H-LC,1.19,239040.0,-7111.0,86317.0,0.39,-2.89,36.11,32.18,16.0,-0.08,1.74,32.11,X200,ATH,IT
41,ITC,452.00,-37.08,57.0,H-LC,2.96,203424.0,3286.0,17596.0,0.23,1.64,8.65,10.44,4.0,0.19,1.48,7.30,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.33,62.0,H-LC,3.73,285152.0,19726.0,37070.0,-0.18,7.43,13.00,21.40,11.0,0.53,2.07,15.05,XY25,NTT,FMCG
20,CIPLA,1795.00,-19.09,57.0,H-LC,5.92,216076.0,11576.0,29840.0,1.05,5.66,13.81,20.25,10.0,0.39,1.57,14.89,X40N,BTT,PHARMA
83,VBL,671.64,-16.73,37.0,H-LC,6.04,300051.0,-15791.0,128452.0,-0.92,-5.00,42.81,35.67,5.0,-0.12,2.18,8.16,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3964.86,67.0,M-SC,7.15,126732.0,-20636.0,72453.0,10.81,-14.00,57.17,35.16,236.0,-0.28,0.92,29.44,XY24,NTT,AUTO
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
58,ROUTE,2236.84,-46.55,48.0,H-SC,18.04,87951.0,-45115.0,137969.0,1.75,-33.90,156.87,69.78,118.0,-0.33,0.64,2.69,SR,ATH,IT
85,WHIRLPOOL,2270.00,-40.03,56.0,M-SC,1.40,101415.0,9916.0,68830.0,1.67,10.84,67.87,86.07,223.0,0.14,0.74,47.27,XR,NTT,DURABLES
33,HONAUT,58357.33,-20.66,50.0,H-SC,2.65,111870.0,-16068.0,63207.0,1.62,-12.56,56.50,36.84,143.0,-0.25,0.81,14.53,X40N,ATH,ELECTRICAL


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.0,241.73,70.0,M-SC,5.44,152646.0,-30432.0,109417.0,-2.90,-16.62,71.68,43.14,230.0,-0.28,1.11,43.59,XY24,NTT,MISC
69,SURYODAY,240.0,40.35,48.0,H-SC,3.29,127939.0,-51132.0,116859.0,-2.43,-28.55,91.34,36.71,135.0,-0.44,0.93,26.94,XR,NTT,BANKS
53,RAJESHEXPO,518.0,1835.61,63.0,L-SC,5.52,52809.0,-84368.0,84463.0,-1.59,-61.50,159.94,0.07,267.0,-1.00,0.38,31.34,OX40N,NTT,JEWELLERY
77,TTKPRESTIG,770.0,101.92,53.0,M-SC,5.01,86971.0,-13806.0,13898.0,-1.37,-13.70,15.98,0.09,245.0,-0.99,0.63,14.26,OX40N,NTT,DURABLES
72,TATAELXSI,9161.0,-20.18,52.0,H-MC,7.97,107996.0,-23839.0,66061.0,-1.19,-18.08,61.17,32.03,98.0,-0.36,0.79,20.96,OX40N,NTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,101.87,63.0,H-SC,12.59,127278.0,-15201.0,34721.0,-0.21,-10.67,27.28,13.70,163.0,-0.44,0.93,50.22,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,101.92,53.0,M-SC,5.01,86971.0,-13806.0,13898.0,-1.37,-13.70,15.98,0.09,245.0,-0.99,0.63,14.26,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,62.0,H-SC,1.98,224910.0,-44757.0,81080.0,1.33,-16.60,36.05,13.47,138.0,-0.55,1.64,13.76,XY24,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.14,145613.0,-30290.0,72311.0,-0.17,-17.22,49.66,23.89,149.0,-0.42,1.06,25.76,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-20.18,52.0,H-MC,7.97,107996.0,-23839.0,66061.0,-1.19,-18.08,61.17,32.03,98.0,-0.36,0.79,20.96,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.12,55.0,H-LC,13.40,294370.0,-51586.0,175445.0,0.24,-14.91,59.60,35.80,1.0,-0.29,2.14,4.28,X40,BTT,IT
39,INFY,2275.00,-16.37,56.0,H-LC,9.45,325272.0,12384.0,159318.0,1.15,3.96,48.98,54.87,3.0,0.08,2.37,10.87,X40,BTT,IT
41,ITC,452.00,-37.08,57.0,H-LC,2.96,203424.0,3286.0,17596.0,0.23,1.64,8.65,10.44,4.0,0.19,1.48,7.30,X40,NTT,FMCG
83,VBL,671.64,-16.73,37.0,H-LC,6.04,300051.0,-15791.0,128452.0,-0.92,-5.00,42.81,35.67,5.0,-0.12,2.18,8.16,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,54.0,H-LC,11.96,250834.0,-10785.0,130007.0,0.15,-4.12,51.83,45.57,7.0,-0.08,1.82,6.73,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,64.0,M-MC,6.95,227183.0,2221.0,165435.0,0.08,0.99,72.82,74.53,192.0,0.01,1.65,33.56,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,8.71,51.0,H-MC,3.94,187560.0,22890.0,45071.0,-0.59,13.90,24.03,41.28,89.0,0.51,1.36,22.99,X40N,ATH,AC
84,VOLTAS,1918.49,0.18,56.0,H-MC,2.97,210210.0,18468.0,77567.0,0.44,9.63,36.90,50.08,99.0,0.24,1.53,17.08,XY25,ATH,AC
24,DABUR,735.00,-3.30,56.0,H-MC,6.17,210308.0,13280.0,76342.0,-0.89,6.74,36.30,45.49,102.0,0.17,1.53,20.66,XY24,BTT,FMCG
18,CAMS,5211.76,-5.16,51.0,H-SC,4.73,107946.0,5942.0,37986.0,0.12,5.83,35.19,43.06,122.0,0.16,0.79,23.94,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.03,56.0,M-SC,1.40,101415.0,9916.0,68830.0,1.67,10.84,67.87,86.07,223.0,0.14,0.74,47.27,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.35,55.0,H-SC,12.27,138378.0,15042.0,118714.0,-0.06,12.20,85.79,108.45,119.0,0.13,1.01,37.27,AR,ATH,MISC
86,WIPRO,420.00,-11.07,54.0,M-LC,5.80,156246.0,5301.0,104154.0,-0.75,3.51,66.66,72.51,53.0,0.05,1.14,9.75,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,62.0,H-MC,5.63,180715.0,907.0,111501.0,-0.51,0.50,61.70,62.52,88.0,0.01,1.31,34.76,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.52,86880.0,-26669.0,66724.0,-0.92,-23.49,76.80,35.27,268.0,-0.40,0.63,112.64,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.51,43.0,H-SC,14.00,87939.0,-13024.0,107558.0,-0.24,-12.90,122.31,93.63,148.0,-0.12,0.64,30.98,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.44,57.0,H-SC,13.13,94920.0,-3060.0,24660.0,0.16,-3.12,25.98,22.05,152.0,-0.12,0.69,37.97,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
82,VALIANTORG,838.00,-324.58,54.0,H-SC,7.79,124899.0,-38706.0,160021.0,0.64,-23.66,128.12,74.15,139.0,-0.24,0.91,60.10,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.52,86880.0,-26669.0,66724.0,-0.92,-23.49,76.80,35.27,268.0,-0.40,0.63,112.64,XR,NTT,HOTELS
8,ATULAUTO,844.00,3964.86,67.0,M-SC,7.15,126732.0,-20636.0,72453.0,10.81,-14.00,57.17,35.16,236.0,-0.28,0.92,29.44,XY24,NTT,AUTO
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
71,TANLA,1963.11,-19.51,74.0,H-SC,9.28,215883.0,-54878.0,384833.0,0.19,-20.27,178.26,121.86,133.0,-0.14,1.57,68.88,AR,ATH,IT
55,RELAXO,1176.00,-31.84,59.0,H-SC,2.19,92872.0,-52288.0,128219.0,-0.04,-36.02,138.06,52.31,136.0,-0.41,0.68,24.31,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,43.53
2,50,73.85


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.53
LC    34.48
MC    22.98
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.98
X40      15.28
X40N     12.96
XR       10.01
XY25      9.76
AR        8.63
OX40N     8.21
X200      1.74
MH        1.71
X5K       1.43
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.40
H-LC    26.27
H-MC    20.04
M-SC    13.58
M-LC     7.14
M-MC     2.63
L-SC     1.55
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.42,-14.02,78.85
FMCG,11.85,-3.84,40.17
FINANCE,9.60,-16.63,62.31
BANKS,7.79,-12.10,63.71
MISC,6.09,-13.52,78.33
PAINTS,5.93,-13.43,38.81
ELECTRICAL,5.58,-7.27,66.39
AC,3.60,2.15,30.98
INSURANCE,3.38,-8.12,45.20


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2861756.0,22
AR,1309909.0,9
XR,1236146.0,13
X40,866854.0,10
X40N,630514.0,10
OX40N,519876.0,10
XY25,409511.0,7
SR,245527.0,2
X5K,120630.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3349355.0,28
H-LC,1380974.0,15
M-SC,1379341.0,17
H-MC,1199507.0,15
M-LC,396991.0,5
M-MC,315540.0,2
L-SC,247663.0,3
L-MC,60125.0,1
L-LC,37798.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1041191.0      6
M-SC       XY24       792093.0      7
H-SC       AR         779128.0      5
           XR         681431.0      6
H-LC       X40        576576.0      5
H-MC       XY24       486212.0      4
H-LC       AR         368941.0      2
M-MC       XY24       315540.0      2
H-SC       X40N       294839.0      4
           SR         245527.0      2
           OX40N      244496.0      4
H-LC       X40N       225212.0      4
H-MC       X40        219464.0      4
           XY25       188295.0      2
L-SC       XR         163200.0      2
M-SC       AR         161840.0      2
M-LC       XY24       161525.0      2
M-SC       OX40N      124856.0      4
M-LC       X5K        120630.0      1
M-SC       XR         115735.0      2
           XY25       114003.0      1
H-MC       XR         111501.0      1
           X40N       110463.0      2
M-LC       XR         104154.0      1
H-LC       X200        86317.0      1
L-SC       OX40N       84463.0      1
M-SC       X40         70814.0      1
H-MC       OX40N       66061.0      1
H-LC       XY24        65195.0      1
H-SC       MH          62743.0      1
L-MC       XR          60125.0      1
H-LC       XY25        58733.0      2
L-LC       XY25        37798.0      1
H-MC       MH          17511.0      1
M-LC       XY25        10682.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
